In [1]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

import os
import pandas as pd
import numpy as np
from typing import Dict, Set, Tuple, List
from collections import defaultdict
import random
import time
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def read_dataframes_from_folder(folder_path: str, file_limit: int) -> List[Tuple[pd.DataFrame, str]]:
    """
    Read dataframes from CSV files in the given folder.

    Args:
        folder_path (str): Path to the folder containing CSV files.
        file_limit (int): Maximum number of files to be read.

    Returns:
        List[Tuple[pd.DataFrame, str]]: A list of tuples containing the dataframes and their corresponding filenames.
    """
    tuple_dataframes = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            if len(tuple_dataframes) < file_limit:
                file_path = os.path.join(folder_path, filename)
                df = pd.read_csv(file_path)
                tuple_dataframes.append((df, os.path.basename(file_path)))
            else:
                break

    return tuple_dataframes

In [20]:
def clean_dataframes(tuple_dataframes: List[Tuple[pd.DataFrame, str]]) -> None:
    """
    Clean the dataframes by renaming columns, dropping the 'Unnamed: 0' column, and dropping empty columns.

    Args:
        tuple_dataframes (List[Tuple[pd.DataFrame, str]]): List of tuples containing the dataframes and their corresponding filenames.
    """
    
    # inner function
    def drop_empty_columns_in_dataframes(dataframes: List[Tuple[pd.DataFrame, str]]) -> None:
        for df, _ in dataframes:
            empty_columns = df.columns[df.isnull().all()].tolist()  # get the list of empty columns
            df.drop(empty_columns, axis=1, inplace=True)  # drop the empty columns
    
    # renaming columns, deleting 'Unnamed: 0', and removing empty dataframes
    indices_to_delete = []
    for index, (df, _) in enumerate(tuple_dataframes):
        df.rename(columns=lambda x: x.replace('col', ''), inplace=True)
        
        if 'Unnamed: 0' in df.columns:
            df.drop('Unnamed: 0', axis=1, inplace=True)  # delete the "Unnamed: 0" column
        
        if df.empty:
            indices_to_delete.append(index)  # mark dataframe for deletion if empty
    
    # reverse sort the indices and delete the empty dataframes
    for index in sorted(indices_to_delete, reverse=True):
        del tuple_dataframes[index]

    # call inner funtion
    drop_empty_columns_in_dataframes(tuple_dataframes)


In [21]:
def drop_columns_without_label(dataframes: List[Tuple[pd.DataFrame, str]], label_df: pd.DataFrame) -> List[Tuple[pd.DataFrame, str]]:
    """
    Drop columns for which no gold standard exists.

    Args:
        dataframes (List[Tuple[pd.DataFrame, str]]): List of tuples containing the dataframes and their corresponding filenames.
        label_df (pd.DataFrame): DataFrame containing the gold standard annotations.

    Returns:
        List[Tuple[pd.DataFrame, str]]: List of tuples containing the cleaned dataframes and their corresponding filenames.
    """
    
    # make a copy
    tuple_dataframes_copy = [(df.copy(), name) for df, name in dataframes]

    # indices of dataframes to delete
    to_delete = []

    # iterate over dataframes
    for table_index, (df, filename) in enumerate(tuple_dataframes_copy):
        for column_index, col in enumerate(df.columns):
            if not any((label_df['table_id'] == filename[:-4] + "_dbpedia") & (label_df['target_column'] == column_index)):
                df.drop(col, axis=1, inplace=True)
        if df.empty:
            to_delete.append(table_index)
        else:
            tuple_dataframes_copy[table_index] = (df, filename)

    # delete empty dataframes from the list in reverse order to prevent index shifting
    for index in reversed(to_delete):
        del tuple_dataframes_copy[index]

    return tuple_dataframes_copy


In [27]:
def sample_columns(tuple_dataframes: List[Tuple[pd.DataFrame, str]], n: int) -> List[Tuple[pd.DataFrame, str]]:
    """
    Sample 'n' columns from all DataFrames combined, delete non-sampled columns, and filter out empty DataFrames.

    Args:
        tuple_dataframes (List[Tuple[pd.DataFrame, str]]): A list of tuples containing the dataframes and their corresponding filenames.
        n (int): Number of columns to sample in total.

    Returns:
        List[Tuple[pd.DataFrame, str]]: A list of tuples containing the non-empty dataframes and their corresponding filenames.
    """
    # add prefixes to ensure unique column names in combined DataFrame
    all_dfs = []
    for idx, (df, _) in enumerate(tuple_dataframes):
        df_prefixed = df.add_prefix(f"df{idx}_")
        all_dfs.append(df_prefixed)

    combined_df = pd.concat(all_dfs, axis=1)

    # sample 'n' columns from the combined DataFrame
    if n >= len(combined_df.columns):
        sampled_columns = combined_df.columns
    else:
        sampled_columns = random.sample(list(combined_df.columns), n)

    # filter combined DataFrame to keep only the sampled columns
    sampled_combined_df = combined_df[sampled_columns]

    filtered_dataframes = []

    # split the combined DataFrame back into individual DataFrames and filter out any empty DataFrames
    for idx, (_, filename) in enumerate(tuple_dataframes):
        relevant_cols = [col for col in sampled_combined_df.columns if col.startswith(f"df{idx}_")]
        df_sampled = sampled_combined_df[relevant_cols].rename(columns=lambda x: x.split("_", 1)[1])
        if not df_sampled.empty:
            filtered_dataframes.append((df_sampled, filename))

    return filtered_dataframes

In [28]:
def create_ground_truth_map(label_df: pd.DataFrame, tuple_dataframes: List[Tuple[pd.DataFrame, str]]) -> Dict[str, Set[int]]:
    """
    Create a ground truth map for evaluation.

    Args:
        label_df (pd.DataFrame): DataFrame containing true labels for evaluation.
        tuple_dataframes (List[Tuple[pd.DataFrame, str]]): List of tuples containing the dataframes and their corresponding filenames.

    Returns:
        Dict[str, Set[int]]: A dictionary mapping label names to sets of global indices that belong to each label.
    """
    ground_truth = defaultdict(set)

    for table_index, (df, filename) in enumerate(tuple_dataframes):
        for new_col_index, old_col_index in enumerate(df.columns):
            label = label_df.query(f"table_id == '{filename[:-4] + '_dbpedia'}' and target_column == {old_col_index}")['annotation_label']

            if label.empty:
                continue

            global_index = sum(df.shape[1] for df, _ in tuple_dataframes[:table_index]) + new_col_index
            ground_truth[label.iloc[0]].add(global_index)

            if len(label.values) != 1:
                raise ValueError("Unexpected number of label values")

    for tuple_dataframe in tuple_dataframes:
        tuple_dataframe[0].columns = range(len(tuple_dataframe[0].columns))

    return ground_truth

In [29]:
def calculate_column_similarity_sbert(dataframe_list, sample_size=12):
    """
    Calculate the similarity between columns of dataframes using Sentence BERT embeddings.

    Parameters:
        dataframe_list (List[pd.DataFrame]): A list of dataframes to be processed.
        sample_size (int, default=100): Number of entries to be used from each column for embeddings.

    Returns:
        pd.DataFrame: A dataframe containing cosine distances between every pair of columns.
    """
    
    # initialize the model
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Create a dict to store the embeddings for each column
    column_embeddings = {}

    # Loop over the dataframes and columns
    count = 0
    for df in dataframe_list:
        for col in df.columns:
            # Sample a subset of entries from the column
            samples = df[col].dropna().astype(str)
            samples = samples.sample(min(len(samples), sample_size))
            
            sample_embeddings = model.encode(samples.tolist())
            column_embeddings[count] = np.mean(sample_embeddings, axis=0)
            count += 1

    # Prepare the embedding matrix
    embedding_matrix = np.vstack(list(column_embeddings.values()))

    # Calculate cosine distance
    distance = 1 - cosine_similarity(embedding_matrix)

    # Convert the distance matrix to a dataframe
    distance_df = pd.DataFrame(distance, index=column_embeddings.keys(), columns=column_embeddings.keys())

    return distance_df

In [30]:
def hierarchical_clustering(distance_matrix: np.ndarray, n_clusters: int) -> Dict[int, Set[int]]:
    """
    Perform hierarchical clustering on the distance matrix.

    Args:
        distance_matrix (np.ndarray): Pairwise distance matrix.
        n_clusters (int): Number of clusters to create.

    Returns:
        Dict[int, Set[int]]: A dictionary mapping cluster labels to sets of indices that belong to each cluster.
    """
    # create an AgglomerativeClustering model
    model = AgglomerativeClustering(n_clusters=n_clusters, metric='precomputed', linkage='average')

    # fit the model to the distance matrix
    model.fit(distance_matrix)

    # get the cluster labels
    labels = model.labels_

    # create a dictionary that maps each cluster label to the set of indices that belong to that cluster
    clusters = defaultdict(set)
    for idx, label in enumerate(labels):
        clusters[label].add(idx)

    return clusters

In [31]:
def evaluate_micro(truth_dict: Dict[str, Set[int]], result_dict: Dict[int, Set[int]]) -> Tuple[float, float]:
    """
    Evaluate clustering performance using micro-average precision, recall.

    Args:
        truth_dict (Dict[str, Set[int]]): Ground truth mapping label names to sets of global indices.
        result_dict (Dict[int, Set[int]]): Resulting clustering mapping cluster labels to sets of indices.

    Returns:
        Tuple[float, float]: Micro-average precision and recall
    """
    # creating the contingency matrix
    contingency_matrix = np.zeros((len(truth_dict), len(result_dict)))

    truth_labels = list(truth_dict.keys())
    result_labels = list(result_dict.keys())

    for i, truth_label in enumerate(truth_labels):
        for j, result_label in enumerate(result_labels):
            # length of intersection between clusters
            # contingency_matrix[i, j] is the number of (global) indices assigned to both clusters
            contingency_matrix[i, j] = len(truth_dict[truth_label] & result_dict[result_label])

    # calculate precision, recall, and f1-score for each label and average them
    total_precision = 0
    total_recall = 0
    total_f1_score = 0
    total_weight = 0

    for i, _ in enumerate(result_labels):
        tp = np.max(contingency_matrix[:, i])  # maximum element in column i (True Positives)
        best_match_index = np.argmax(contingency_matrix[:, i])  # get the index of the best match (TP - index)
        tp_plus_fp = np.sum(contingency_matrix[best_match_index, :])  # sum of elements in row of best match (TP + FP)
        tp_plus_fn = np.sum(contingency_matrix[:, i])  # sum of elements in column i (TP + FN)

        weight = tp_plus_fn  # the weight for each label is the total number of true positives plus false negatives

        if tp_plus_fp > 0:
            precision = tp / tp_plus_fp
            total_precision += weight * precision
        if tp_plus_fn > 0:
            recall = tp / tp_plus_fn
            total_recall += weight * recall

        total_weight += weight

    avg_precision = total_precision / total_weight
    avg_recall = total_recall / total_weight

    return avg_precision, avg_recall


In [33]:
def execute_script() -> dict:
    metrics = {}
    
    # path to the folder containing CSV files 
    base_dir = os.path.dirname(os.path.abspath("Baseline.ipynb"))
    folder_path = os.path.join(base_dir, "..","Data", "GitTables", "tables")
    
    # maximum number of files to be read
    file_limit = 1500
    
    # read dataframes from the folder
    tuple_dataframes = read_dataframes_from_folder(folder_path, file_limit)
    
    # reading in true labels for evaluation
    base_dir = os.path.dirname(os.path.abspath("Baseline.ipynb"))
    label_path = os.path.join(base_dir, "..", "Data", "GitTables", "dbpedia_gt.csv")
    label_df = pd.read_csv(label_path)

    
    clean_dataframes(tuple_dataframes)
    
    # Clean the dataset and drop columns without gold standard annotations
    tuple_dataframes_copy = drop_columns_without_label(tuple_dataframes, label_df)
    
    n = 2000
    random_sample = sample_columns(tuple_dataframes_copy, 2000)
    
    ground_truth_map = create_ground_truth_map(label_df, random_sample)
    
    # extract dataframes from the tuple list
    dataframes = [df for df, _ in random_sample]
    
    # calculate column similarity using Sentence-BERT embeddings
    s = 12
    d = calculate_column_similarity_sbert(dataframes, sample_size=s)
    
    # clustering hierarchically
    num_hierarchical_clusters = len(ground_truth_map.keys())

    cl = hierarchical_clustering(d, num_hierarchical_clusters)

    # evaluate clustering performance
    precision, recall = evaluate_micro(ground_truth_map, cl)
    metrics['precision'] = precision
    metrics['recall'] = recall
    metrics['f1_score'] = 2*(precision*recall)/(precision + recall)
    metrics['calculated_embeddings'] = (0.5 * n*(n-1))*s

    return metrics

In [36]:
# EXECUTE HERE

# ADJUST ACCORDING TO PREFERENCE
# FOR STABLE RESULT CONISDER num_iteraitons = 10,
# FOR QUICK EXECUTION num_itertaions = 1
num_iterations = 2

results = [execute_script() for _ in range(num_iterations)]
averages = {key: sum([result[key] for result in results]) / num_iterations for key in results[0]}
print(averages)

{'precision': 0.5123353912265673, 'recall': 0.5959982283246595, 'f1_score': 0.5508969850551932, 'calculated_embeddings': 23988000.0}


In [37]:
# RESULTS AS USED IN THESIS, DO NOT EXECUTE

In [81]:
results = [execute_script() for _ in range(10)]
averages = {key: sum([result[key] for result in results]) / 10 for key in results[0]}
print(averages)

{'initial_columns': 18578.0, 'algorithm_runtime': 37.654068326950075, 'precision': 0.5248148482539488, 'recall': 0.5840061962309203, 'f1_score': 0.551536919780547, 'calculated_embeddings': 19990000.0}
